## Hardware and software hierarchies
CUDA is a parallel computing platform and application programming interface ment for general purpose computing on graphics processing unit(GPGPU). CUDA is based on C programming, its compiler is called NVCC.
![NVCC](NVCC.png)
In CUDA programming host refers to the CPU and its global memory and device refers to the GPU and its global memory. The structure of a GPU hardware is hierarchical, it has 4 main levels, GPU, GPU has many streaming multi processors(SM), each SM is divided into multiple partitions and the partitions are made of cores. So the software also needs a similar level structure for execution, it has the below hierarchical structure.
* Grid : Each kernel(GPU function) is executed in a grid. A grid has multiple blocks.
* Block/Thread block : The kernel(function) that needs to be executed on the GPU is divided into blocks. A block corresponds to a SM. So the first step for a kernel execution is to configure the number of blocks needed for that kernel i.e it configures the number of SM's needed to execute that kernel. GigaThread unit is responsible for efficiently assigning blocks to SM's.
* Warp : Each block is sub divided into warps. Each SM partition will get a fraction of these warps. For example if each block has 32 warps and the SM has 4 partions then each partition will get 8 warps. Warp scheduler unit(present in each partition) is responsible to manage the warps.
* Thread : A warp contains 32 threads, each thread will be executed on a seperate core. Number of threads with in a block is another key paramater to be configured for the kernel execution. We can get the number of warps within a block by dividing the number of threads by 32.

## Hello world program
Include the cuda and stdio headers. Add a kernel(test01) and a function(main). A kernel is a function that is going to be executed on the GPU, use the __global__ specifier to define a kernel. The normal function will be executed on the CPU. A kernel call is like a function with <<< syntax to give the kernel parameters.
```
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>

__global__ void test01()
{
    //print the blocks and the threads IDs
    int warp_id = 0;
    warp_id = threadIdx.x/32;  //since each warp has 32 threads
    printf("\n The block ID is %d --- The thread ID is %d -- The warp ID is %d", blockIdx.x, threadIdx.x, warp_id);
}

int main()
{
    //kernel execution - kernel_name<<<num_of_blocks, num_of_threads_per_block>>>();
    test01 <<<2, 8>>> ();
    //Cpu will wait for the GPU to complete the kernel execution
    cudaDeviceSynchronize();

    return 0;
}
```
blockIdx and threadIdx are predefined variables which contain block ID and thread ID. The CUDA numbers(parameters) can be distributed in different dimensions(x,y and z) based on the requirement. Use nvcc to build the application, on windows nvcc will be integrated to visaul studio community so it can be built directly in IDE. In Linux we can use the nvcc commands to build the code. On successful build we can run the application. The 2 kernel parameters are number of blocks and number of threads per block, these values should be powers of 2. The prints of this program will be random, as any thread on any block can be completed in any order as they are all running in parallel. The number of blocks and number of threads per block cannot exceed the maximum values supported by the GPU, for example if your local GPU supports a max of 1024 threads per block and if you give it a value of 2048 then the kernel execution will fail. We can get the maximum values support by the GPU from its white paper. So in this case if we want to run 2048 threads we have to use a minimum of 2 blocks. cudaDeviceSynchronize() method is used for syncronization between CPU and GPU, CPU will wait at the method for the GPU to complete the execution of the kernel and return.

## Compiling CUDA on Linux
* Command to get the verison of the nvcc installed : nvcc --version
* nvcc -o project001 project001.cu : Command to compile a cuda file(project001.cu in this example command). -o option is used to specify the name of the output executable file.
* ./project001 : Run the executable

## Vector addition
This program adds 2 vectors with 1024 elements each by index. We choose to use 1 block and 1024 threads per block. This choice aligns the number of threads with the number of elements in the vector, thus we can use the thread id's to access the elements in the vector. All addition will be done in parallel on 1024 cores. Below are the essential steps for any CUDA program.
* Allocate memory for input and output on the host(CPU) and device(GPU). For this program allocate memory for vectors A,B,C on the host and the decice.
* Initialize the inputs on the CPU. In this case initialize vectors A and B on the CPU.
* Copy the inputs to the device. Copy the vectors A and B to the GPU, as the addition will be done on the GPU.
* Once the inputs are on the device, define the kernel to be executed. In this case a 'vectorAdd' kernel which adds the device vectors A and B elementwise and saves the results to device vector C.
* Launch the kernel with suitable number of blocks and threads per block giving it the device parameters(kernel function parameters). For this program 1 block and 1024 threads per block.
* The kernel stores the results in the GPU memory, so this next step is to copy the results back to the CPU. Copy device vector C to host vector C. This result can then be used for furthur computation as per the program requirement.
* The final step is to free the allocated memory on both the host and the device.

```
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <memory>
#include <stdio.h>

#define SIZE 1024  //Define the size of the vectors

//kernel for vector addition
__global__ void vectorAdd(int *A, int *B, int *C, int n)
{
    int i = threadIdx.x;
    C[i] = A[i] + B[i];
}

int main()
{
    int *A, *B, *C;  //host vectors
    int *d_A, *d_B, *d_C;  //device vectors
    int size = SIZE * sizeof(int);

    //allocate host vectors on the CPU memory
    A = (int*)malloc(size);
    B = (int*)malloc(size);
    C = (int*)malloc(size);

    //allocate device vectors on the GPU memory
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    //initialize the host vectors
    for (int i = 0; i < SIZE; i++)
    {
        A[i] = i;
        B[i] = SIZE - i;
    }

    //copy host vectors to device
    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);

    //Execute the kernel
    vectorAdd << <1, 1024 >> > (d_A, d_B, d_C, SIZE);
    
    //copy result back to the host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    printf("\nExecution finished\n");
    for (int i = 0; i < SIZE; i++)
    {
        printf("%d + %d = %d", A[i], B[i], C[i]);
        printf("\n");
    }

    //cleanup
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(A);
    free(B);
    free(C);

    return 0;
}
```

Now say that we want to add vectors of sizes 2048 and maximum threads supported by a block is 1024 for your GPU. To solve this we have to use more than 1 block, the number of vector elements must match the total number of threads run by the kernel. The number of blocks * number of threads per block, will give the total threads that will be run by the kernel. For this case we can use 2 blocks with each having 1024 threads. This will result in the vector index calculation in the kernel to use the blockid along with the threadid, int i = threadIdx.x + blockIdx.x * blockDim.x(number_of_threads_per_block). Use the predefined variable blockDim to get the number of threads per block in the x direction. Below is the updated kernel and its call.
```
//kernel for vector addition
__global__ void vectorAdd(int* A, int* B, int* C, int n)
{
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    C[i] = A[i] + B[i];
}

//Execute the kernel
vectorAdd << <2, 1024 >> > (d_A, d_B, d_C, SIZE);
```
How many SM's will be used to execute this kernel and how will it impact the performance. Since we are using 2 blocks only 2 SM's will be used for the execution of this kernel. If the total number of the SM's on the GPU are 108 then our GPU utilization is less than 2%. This GPU utilization is a key indicator of the GPU performance. In order to increase the SM's we can increase the block count while decreasing the number of threads per block. The minimum number of threads that can be used in a block is 32(1 warp), which will result 64 blocks being used, thus using 64 SM's and increasing the GPU utilization.
We can validate the performance improvement by measuring the execution times in both the cases. We can calculate the execution times using 2 ways, one way is to use CUDA's API's and the other is to use the profilers(Nsight systems and Nsight compute), here we will check with the CUDA API method.  
We can create CUDA events and record as needed to measure time in code as shown below.
```
#include <cuda.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <memory>
#include <stdio.h>

#define SIZE 2048  //Define the size of the vectors

//kernel for vector addition
__global__ void vectorAdd(int* A, int* B, int* C, int n)
{
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    C[i] = A[i] + B[i];
}

int main()
{
    int* A, * B, * C;  //host vectors
    int* d_A, * d_B, * d_C;  //device vectors
    int size = SIZE * sizeof(int);

    //allocate host vectors on the CPU memory
    A = (int*)malloc(size);
    B = (int*)malloc(size);
    C = (int*)malloc(size);

    //allocate device vectors on the GPU memory
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    //initialize the host vectors
    for (int i = 0; i < SIZE; i++)
    {
        A[i] = i;
        B[i] = SIZE - i;
    }

    //copy host vectors to device
    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);

    //create events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    //Execute the kernel and time
    cudaEventRecord(start);
    vectorAdd << <64, 32 >> > (d_A, d_B, d_C, SIZE);
    cudaEventRecord(stop);

    //copy result back to the host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Execution time : %f milliseconds\n", milliseconds);

    //cleanup
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(A);
    free(B);
    free(C);

    return 0;
}
```
Change the number of blocks and the threads per block and measure the timings, we will get the best time when we use 64 blocks and 32 threads per block. The better executuion times is due to the samller block size that can be better managed by a single SM.

## Extra large vector addition
Here we are going to look at adding two vectors with more than 400 million elements each. We are going to use 1024 threads per block(which is the maximum threads per block) and 1024*432 blocks to support this addition. Below is the code.
```
#include <cuda.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <memory>
#include <stdio.h>

#define SIZE 1024*432*1024  //Define the size of the vectors, more tha 400 million elements

//kernel for vector addition
__global__ void vectorAdd(int* A, int* B, int* C, int n)
{
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    C[i] = A[i] + B[i];
}

void random_ints(int *x, int size)
{
    for (int i = 0; i < size; i++)
    {
        x[i] = rand() % 100;
    }
}

int main()
{
    int* A, * B, * C;  //host vectors
    int* d_A, * d_B, * d_C;  //device vectors
    int size = SIZE * sizeof(int);

    //allocate host vectors on the CPU memory
    A = (int*)malloc(size);
    B = (int*)malloc(size);
    C = (int*)malloc(size);

    //allocate device vectors on the GPU memory
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    //initialize the host vectors
    random_ints(A, SIZE);
    random_ints(B, SIZE);

    //copy host vectors to device
    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);

    //Execute the kernel
    //num_of_blocks * num_of_threads_pre_block should be equal to the number of elements in each vector
    vectorAdd << <1024*432, 1024 >> > (d_A, d_B, d_C, SIZE);
    
    cudaDeviceSynchronize();

    //copy result back to the host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    printf("\nFinished\n");

    //print the first 100 elements
    for (int i = 0; i < 100; i++)
    {
        printf("\nElement ID = %d ---> %d + %d = %d", i, A[i], B[i], C[i]);
    }

    //cleanup
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(A);
    free(B);
    free(C);

    return 0;
}
```
This will work as long as we have enough RAM on the CPU. Now say we increased the vector size to 1024*1024*1204(over a billion elements), this might result in not enough memeory(RAM) on the CPU as 3 vectors of this size are being created, and each vector is of size 4*1024*1024*1024 which is 4GB. Similarly we have to lookout for the memory on the GPU too. The solution is to divide the vectors into chunks and process a chunk at a time.
```
#include <cuda.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <memory>
#include <stdio.h>

#define TOTAL_SIZE 1024*1024*1024  //Define the size of the vectors, more than 1 billion elements
#define CHUNK_SIZE 1024*1024*128   //Each chunk size, total vector divided into 8 pieces
#define BLOCK_SIZE 1024            //Number of threads per block

//kernel for vector addition
__global__ void vectorAdd(int* a, int* b, int* c, int chunkSize)
{
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i < chunkSize) //Ensure we do not go out of bounds
    {
        c[i] = a[i] + b[i];
    }
}

void random_ints(int *x, int size)
{
    for (int i = 0; i < size; i++)
    {
        x[i] = rand() % 100;
    }
}

int main()
{
    int* chunk_a, * chunk_b, * chunk_c;  //host vectors
    int* d_a, * d_b, * d_c;  //device vectors
    int chunkSizeInBytes = CHUNK_SIZE * sizeof(int);

    //allocate host vectors on the CPU memory
    chunk_a = (int*)malloc(chunkSizeInBytes);
    chunk_b = (int*)malloc(chunkSizeInBytes);
    chunk_c = (int*)malloc(chunkSizeInBytes);

    //allocate device vectors on the GPU memory
    cudaMalloc((void**)&d_a, chunkSizeInBytes);
    cudaMalloc((void**)&d_b, chunkSizeInBytes);
    cudaMalloc((void**)&d_c, chunkSizeInBytes);

    //calculate the number of blocks needed for the kernel to execute
    int numBlocks = (CHUNK_SIZE + BLOCK_SIZE - 1) / BLOCK_SIZE;

    //loop through chunk by chunk and execute the kernel
    for (long long offset = 0; offset < TOTAL_SIZE; offset += CHUNK_SIZE)
    {
        int currentChunkSize = (TOTAL_SIZE - offset) < CHUNK_SIZE ? TOTAL_SIZE - offset : CHUNK_SIZE;
        printf("\nOffset %d\n", offset);

        //populate the chunks with random data
        random_ints(chunk_a, CHUNK_SIZE);
        random_ints(chunk_b, CHUNK_SIZE);

        //copy chunks to the GPU
        cudaMemcpy(d_a, chunk_a, chunkSizeInBytes, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, chunk_b, chunkSizeInBytes, cudaMemcpyHostToDevice);

        //launch the kernel
        vectorAdd << <numBlocks, BLOCK_SIZE >> > (d_a, d_b, d_c, CHUNK_SIZE);

        //copy result back to the host chunk
        cudaMemcpy(chunk_c, d_c, chunkSizeInBytes, cudaMemcpyDeviceToHost);

        //In real applications chunk_c will be used for furthur processing
        //print the first 5 elements of every chunk
        for (int i = 0; i < 5; i++)
        {
            printf("\nElement ID = %d ---> %d + %d = %d", i, chunk_a[i], chunk_b[i], chunk_c[i]);
        }
    }

    printf("\nFinished\n");

    //cleanup
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(chunk_a);
    free(chunk_b);
    free(chunk_c);

    return 0;
}
```